In [2]:
import sys
sys.path.append('../') 

from config.reddit import REDDIT_CL_ID, REDDIT_CL_SECRET

import praw
import pandas as pd

In [3]:
user_agent = "scrapper 1.0"
# Credenciales de la aplicación
client_id = REDDIT_CL_ID
client_secret = REDDIT_CL_SECRET

# Crear una instancia de Reddit
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

In [4]:
df_publicaciones = pd.DataFrame(columns=['id', 'titulo', 'autor', 'fecha_submission'])
df_comentarios = pd.DataFrame(columns=['id_publicacion', 'usuario', 'fecha', 'comentario'])

In [5]:
for submission in reddit.subreddit('Twitch').hot(limit=100):
    submission_id = submission.id
    title = submission.title
    author = submission.author
    submission_date = submission.created_utc
    url = submission.url

    # Guardar los detalles de la publicación en el dataframe correspondiente
    df_nueva_publicacion = pd.DataFrame({'id': [submission_id],
                                        'titulo': [title],
                                        'autor': [author],
                                        'fecha_submission': [submission_date],
                                        'url': [url]})
    df_publicaciones = pd.concat([df_publicaciones, df_nueva_publicacion], ignore_index=True)

    # Reemplazar los objetos MoreComments con los comentarios reales
    submission.comments.replace_more(limit=None)
    comments = submission.comments.list()

    # Recorrer los comentarios de la publicación
    for comment in comments:
        comentario_submission_id = submission_id
        usuario = comment.author.name if comment.author else "N/A"
        fecha = comment.created_utc
        comentario = comment.body
        
        # Guardar los comentarios en el dataframe correspondiente
        df_nuevo_comentario = pd.DataFrame({'id_publicacion': [comentario_submission_id],
                                            'usuario': [usuario],
                                            'fecha': [fecha],
                                            'comentario': [comentario]})
        df_comentarios = pd.concat([df_comentarios, df_nuevo_comentario], ignore_index=True)


In [6]:
df_publicaciones

,id,titulo,autor,fecha_submission,url
0,hi9i18,!!Read Before Posting!!,Havryl,1.593467e+09,https://www.reddit.com/r/Twitch/comments/hi9i1...
1,1c33y2i,Collaboration / LFG Thread,AutoModerator,1.713020e+09,https://www.reddit.com/r/Twitch/comments/1c33y...
2,1c4ji7x,Has this happened to anyone else before? I'm u...,splatbob1,1.713179e+09,https://i.redd.it/6dvpcr2klmuc1.jpeg
3,1c4qgy8,Small esports tournament scoreboard. help needed.,thatmcgboy,1.713198e+09,https://www.reddit.com/r/Twitch/comments/1c4qg...
4,1c3vkc9,How can I support my Girlfriend?,Kiseji,1.713107e+09,https://www.reddit.com/r/Twitch/comments/1c3vk...
...,...,...,...,...,...
95,1c2oyq9,What's the deal with channels that seem to alw...,bobombpom,1.712968e+09,https://www.reddit.com/r/Twitch/comments/1c2oy...
96,1c2o4mn,Affiliate Onboarding ID Verification,HispanicattheDiscoPR,1.712966e+09,https://www.reddit.com/r/Twitch/comments/1c2o4...
97,1c2uvfn,I got a capture card but can't figure out how ...,uninvitedwhitechick,1.712988e+09,https://www.reddit.com/r/Twitch/comments/1c2uv...
98,1c2lxo3,Twitch streamers without Discord,LongDnTattoos,1.712960e+09,https://www.reddit.com/r/Twitch/comments/1c2lx...


In [7]:
df_comentarios

,id_publicacion,usuario,fecha,comentario
0,1c33y2i,TerrifyingT,1.713023e+09,"Oh neat!\n\nI'm Tessa\nI'm 43\nCanada\nMon, We..."
1,1c33y2i,splatbob1,1.713181e+09,Splatbob1 \n 22 \n US (East Coas...
2,1c4ji7x,Kitchen_Inevitable_4,1.713183e+09,Any interaction that doesn't happen in your ch...
3,1c4ji7x,PhuzzyBond,1.713185e+09,Seems like he's got too much time on his hands...
4,1c4ji7x,caramel-syrup,1.713181e+09,that is so condescending
...,...,...,...,...
1213,1c2lqwp,Kougeru-Sama,1.712966e+09,> very low activity on the chat\n\nprobably b...
1214,1c2lqwp,M694V80,1.712962e+09,No I had one yesterday and it's not like like ...
1215,1c2lqwp,M694V80,1.712967e+09,had to kill stream and restart that did the tr...
1216,1c2lqwp,Reiker0,1.712973e+09,Just ignore it and move on.\n\nImagine if you ...
